In [34]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split,RandomizedSearchCV,cross_val_score,GridSearchCV
from sklearn.utils import resample
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [35]:
other_sports=[
 'Nordic Combined',
 'Freestyle Skiing',
 'Snowboarding',
 'Synchronized Swimming',
 'Baseball',
 'Rhythmic Gymnastics',
 'Taekwondo',
 'Beach Volleyball',
 'Triathlon',
 'Softball',
 'Curling',
 'Rugby Sevens',
 'Skeleton',
 'Trampolining',
 'Golf']

In [36]:
df=pd.read_csv("../data/df_2classes.csv", index_col=[0])
df["medal"]=df["medal"].fillna("No medal")

#df=df[df["season"]=="Summer"]
#df["sport"]=df["sport"].apply(lambda x: x if x not in other_sports else "Other sports")
medal_df=df[["id","medal"]]
df.drop(["sport","medal"],axis=1,inplace=True)

In [37]:
df

,id,sex,age,height,weight,noc,year,season,city,event
35564,18289,F,18.0,175.0,69.0,ANG,2008,Summer,Beijing,Handball Women's Handball
112673,57077,F,27.0,164.0,69.0,POL,1980,Summer,Moskva,"Canoeing Women's Kayak Singles, 500 metres"
15820,8528,M,24.0,171.0,62.0,ROU,2012,Summer,London,Gymnastics Men's Pommelled Horse
164015,82322,M,22.0,195.0,87.0,FRA,2004,Summer,Athina,Rowing Men's Coxed Eights
19092,10102,F,18.0,160.0,43.0,GER,2000,Summer,Sydney,Rhythmic Gymnastics Women's Group
...,...,...,...,...,...,...,...,...,...,...
271078,135553,F,25.0,168.0,80.0,URS,1956,Summer,Melbourne,Athletics Women's Shot Put
271080,135553,F,33.0,168.0,80.0,URS,1964,Summer,Tokyo,Athletics Women's Shot Put
271082,135554,M,28.0,182.0,82.0,POL,1980,Summer,Moskva,"Fencing Men's Foil, Team"
271102,135563,F,19.0,171.0,64.0,RUS,2000,Summer,Sydney,Athletics Women's 4 x 400 metres Relay


In [38]:
medal_df.reset_index(drop=True)

,id,medal
0,18289,No medal
1,57077,No medal
2,8528,No medal
3,82322,No medal
4,10102,No medal
...,...,...
60947,135553,Medal
60948,135553,Medal
60949,135554,Medal
60950,135563,Medal


In [39]:
numeric_df=df.select_dtypes(exclude="object")
cat_df=df.select_dtypes(include="object")

In [40]:
cat_df

,sex,noc,season,city,event
35564,F,ANG,Summer,Beijing,Handball Women's Handball
112673,F,POL,Summer,Moskva,"Canoeing Women's Kayak Singles, 500 metres"
15820,M,ROU,Summer,London,Gymnastics Men's Pommelled Horse
164015,M,FRA,Summer,Athina,Rowing Men's Coxed Eights
19092,F,GER,Summer,Sydney,Rhythmic Gymnastics Women's Group
...,...,...,...,...,...
271078,F,URS,Summer,Melbourne,Athletics Women's Shot Put
271080,F,URS,Summer,Tokyo,Athletics Women's Shot Put
271082,M,POL,Summer,Moskva,"Fencing Men's Foil, Team"
271102,F,RUS,Summer,Sydney,Athletics Women's 4 x 400 metres Relay


In [41]:
knn_imp = KNNImputer(n_neighbors=5)
numeric_df = pd.DataFrame(knn_imp.fit_transform(numeric_df),
                           columns=numeric_df.columns)

In [42]:
cat_cols=cat_df.columns
cat_dummy_encoder = OneHotEncoder(handle_unknown='ignore')
df_cat_ohe = cat_dummy_encoder.fit_transform(cat_df).toarray()
cat_ohe_cols = cat_dummy_encoder.get_feature_names(cat_cols)
df_cat_ohe = pd.DataFrame(df_cat_ohe,
                               columns=cat_ohe_cols)

In [43]:
df_cat_ohe

,sex_F,sex_M,noc_AFG,noc_AHO,noc_ALB,noc_ALG,noc_AND,noc_ANG,noc_ANT,noc_ARG,...,"event_Wrestling Men's Super-Heavyweight, Freestyle","event_Wrestling Men's Super-Heavyweight, Greco-Roman","event_Wrestling Men's Welterweight, Freestyle","event_Wrestling Men's Welterweight, Greco-Roman","event_Wrestling Women's Featherweight, Freestyle","event_Wrestling Women's Flyweight, Freestyle","event_Wrestling Women's Heavyweight, Freestyle","event_Wrestling Women's Light-Heavyweight, Freestyle","event_Wrestling Women's Lightweight, Freestyle","event_Wrestling Women's Middleweight, Freestyle"
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60947,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60948,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60949,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60950,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
df=pd.concat([numeric_df,df_cat_ohe],axis=1)

In [45]:
df.reset_index(drop=True)

,id,age,height,weight,year,sex_F,sex_M,noc_AFG,noc_AHO,noc_ALB,...,"event_Wrestling Men's Super-Heavyweight, Freestyle","event_Wrestling Men's Super-Heavyweight, Greco-Roman","event_Wrestling Men's Welterweight, Freestyle","event_Wrestling Men's Welterweight, Greco-Roman","event_Wrestling Women's Featherweight, Freestyle","event_Wrestling Women's Flyweight, Freestyle","event_Wrestling Women's Heavyweight, Freestyle","event_Wrestling Women's Light-Heavyweight, Freestyle","event_Wrestling Women's Lightweight, Freestyle","event_Wrestling Women's Middleweight, Freestyle"
0,18289.0,18.0,175.0,69.0,2008.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,57077.0,27.0,164.0,69.0,1980.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8528.0,24.0,171.0,62.0,2012.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,82322.0,22.0,195.0,87.0,2004.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10102.0,18.0,160.0,43.0,2000.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60947,135553.0,25.0,168.0,80.0,1956.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60948,135553.0,33.0,168.0,80.0,1964.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60949,135554.0,28.0,182.0,82.0,1980.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60950,135563.0,19.0,171.0,64.0,2000.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
df["medal"]=medal_df.medal.values

In [47]:
medal_df.medal.values

array(['No medal', 'No medal', 'No medal', ..., 'Medal', 'Medal', 'Medal'],
      dtype=object)

In [48]:
df

,id,age,height,weight,year,sex_F,sex_M,noc_AFG,noc_AHO,noc_ALB,...,"event_Wrestling Men's Super-Heavyweight, Greco-Roman","event_Wrestling Men's Welterweight, Freestyle","event_Wrestling Men's Welterweight, Greco-Roman","event_Wrestling Women's Featherweight, Freestyle","event_Wrestling Women's Flyweight, Freestyle","event_Wrestling Women's Heavyweight, Freestyle","event_Wrestling Women's Light-Heavyweight, Freestyle","event_Wrestling Women's Lightweight, Freestyle","event_Wrestling Women's Middleweight, Freestyle",medal
0,18289.0,18.0,175.0,69.0,2008.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No medal
1,57077.0,27.0,164.0,69.0,1980.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No medal
2,8528.0,24.0,171.0,62.0,2012.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No medal
3,82322.0,22.0,195.0,87.0,2004.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No medal
4,10102.0,18.0,160.0,43.0,2000.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No medal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60947,135553.0,25.0,168.0,80.0,1956.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Medal
60948,135553.0,33.0,168.0,80.0,1964.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Medal
60949,135554.0,28.0,182.0,82.0,1980.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Medal
60950,135563.0,19.0,171.0,64.0,2000.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Medal


In [49]:
df.to_csv("../data/event2-with-city.csv")